In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [6]:
def get_edges(line):
    edge = line.strip().split(',')
    n1 = edge[0]
    n2 = edge[1]
    if n1 < n2:
         return (n1,n2)
    elif n1 > n2:
         return (n2,n1)
    else:
        pass #n1 == n2

def lista_aristas_dep(tupla):
    aristas = []
    for i in range(len(tupla[1])):
        aristas.append(((tupla[0],tupla[1][i]),'exists'))
        for j in range(i+1,len(tupla[1])):
            if tupla[1][i] < tupla[1][j]:
                aristas.append(((tupla[1][i],tupla[1][j]),('pending',tupla[0])))
            else:
                aristas.append(((tupla[1][j],tupla[1][i]),('pending',tupla[0])))
    return aristas

def filtro(tupla):
    return (len(tupla[1]) >=2 and 'exists' in tupla[1])

def triciclos(h):
    l = []
    for i in h[1]:
        if i != 'exists':
            l.append((i[1],h[0][0],h[0][1]))
    return l

def principal(sc,files):
    rdd = sc.parallelize([])
    for file_name in files:    
        file_rdd = sc.textFile(file_name)
        rdd = rdd.union(file_rdd)
    fin = rdd.map(get_edges).filter(lambda x: x is not None).distinct()
    adj = fin.groupByKey().mapValues(list).flatMap(lista_aristas_dep).\
        groupByKey().mapValues(list).filter(filtro).flatMap(triciclos)
    return adj.collect()

In [7]:
principal(sc,["g1.txt","g4.txt"])

[('B', <pyspark.resultiterable.ResultIterable at 0x7fb0346b45e0>),
 ('C', <pyspark.resultiterable.ResultIterable at 0x7fb0346b46d0>),
 ('D', <pyspark.resultiterable.ResultIterable at 0x7fb0346b4760>),
 ('A', <pyspark.resultiterable.ResultIterable at 0x7fb0346b47f0>),
 ('E', <pyspark.resultiterable.ResultIterable at 0x7fb0346b4820>)]